In [1]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import requests
import os
import alpaca_trade_api as tradeapi
import requests

%matplotlib inline

In [2]:
load_dotenv()

True

In [4]:
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

#verify
type(alpaca_api_key)
type(alpaca_secret_key)

str

In [67]:
#Create the Alpaca API object
alpaca = tradeapi.REST?

#define it with the parameters
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2",
    base_url='https://paper-api.alpaca.markets' # WHAT DOES THIS MEAN?
)

Init signature:
tradeapi.REST(
    key_id: str = None,
    secret_key: str = None,
    base_url: alpaca_trade_api.common.URL = None,
    api_version: str = None,
    oauth=None,
    raw_data: bool = False,
)
Docstring:      <no docstring>
Init docstring:
:param raw_data: should we return api response raw or wrap it with
                 Entity objects.
File:           /opt/anaconda3/envs/dev/lib/python3.7/site-packages/alpaca_trade_api/rest.py
Type:           type
Subclasses:     


In [69]:
#Format Current date as ISO format
start = pd.Timestamp('2000-05-14', tz = 'America/New_York').isoformat()
end = pd.Timestamp('2023-01-14', tz = 'America/New_York').isoformat()

In [74]:
active_assets = alpaca.list_assets()  # you could leave out the status to also get the inactive ones

Elimate all that aren't on the NYSE:

In [76]:
# sp = [x for x in active_assets if x.exchange == 'NYSE']

z = []

for x in active_assets:
    if x.exchange =='NYSE':
        z.append(x)

Append all the tickers into a list, y:

In [81]:
y = []

for i in z:
    y.append(i.symbol)


The length of our first investment universe:

In [82]:
len(y)

3679

In [80]:
y

['ARA',
 'DHR.PRA',
 'LEAF',
 'BRSS',
 'BK.PRC',
 'HOME',
 'PSXP',
 'NETS',
 'BDX.WI',
 'BEL',
 'PAY',
 'FLY',
 'KMM',
 'VGR.WI',
 'NBD_DELISTED',
 'EDR',
 'AIV.WI',
 'KCAC.U',
 'BXP.PRB',
 'BID',
 'TIER',
 'FCRW',
 'P',
 'RTP.U',
 'BMRG.U',
 'NEE.PRI',
 'IPOE.U',
 'WFC.PRO',
 'GS.PRN',
 'QTS.PRA',
 'C.PRS',
 'BSN.U',
 'IRR',
 'NYRT',
 'SFS',
 'COF.PRH',
 'HIFR',
 'JHD',
 'IIAC.U',
 'BFT.U',
 'HGH',
 'PRE.PRG',
 'SRE.PRA',
 'NFC',
 'INF',
 'SPNV.U',
 'ADF.U',
 'LOAK.U',
 'LCM',
 'VIV',
 'IACA.U',
 'SFTW.U',
 'GPM',
 'WRB.PRB',
 'NAV.PRD',
 'MDP',
 'ATCO.PRG',
 'NTX',
 'BAF',
 'SWI.WI',
 'OPP.RT',
 'KSU.PR',
 'ANH',
 'IVR.PRA',
 'AJAX.U',
 'HCXZ',
 'CHU',
 'ALUS.U',
 'MNE',
 'FIT',
 'ATC',
 'MCX',
 'WGP',
 'AIW',
 'COL_DELISTED',
 'JBR',
 'CCI.PRA',
 'PUK.WI',
 'TSS',
 'SNX.WI',
 'PRSP',
 'BBU.WI',
 'PUK.PR',
 'AET_DELISTED',
 'CNR',
 'RNR.PRE',
 'MUH',
 'CLGX',
 'NPN',
 'SC',
 'REXR.PRA',
 'DCUD',
 'VEDL',
 'NSH.U',
 'IFFT',
 'FSKR',
 'PE',
 'NEE.PRK',
 'HMLP',
 'HZAC.U',
 'IVAN.U',
 '